# Solution-1 using python sqlite3


In [32]:
#importing required libraries
import sqlite3 as s
import csv

In [33]:
#creating the connection and creating database
connection = s.connect('C:\\Users\\Dinesh ram\\Downloads\\Data Engineer - Assignment Database.db')
cur = connection.cursor()

In [34]:
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
#fetch all table names
tables = cur.fetchall()
for table in tables:
    table_name = table[0]
    print("Table:", table_name)   
    cur.execute("PRAGMA table_info('{}')".format(table_name))  
    #fetch all column info
    columns = cur.fetchall()
    #print column names
    for column in columns:
        print("Column:", column[1])
    print() 

Table: sqlite_sequence
Column: name
Column: seq

Table: sales
Column: sales_id
Column: customer_id

Table: orders
Column: order_id
Column: sales_id
Column: item_id
Column: quantity

Table: items
Column: item_id
Column: item_name

Table: customers
Column: customer_id
Column: age



In [35]:
'''extract the total quantities of each item bought per customer aged 18-35.
- For each customer, get the sum of each item
- Items with no purchase (total quantity=0) should be omitted from the final
list'''

command = '''select c.customer_id as customer,c.age as Age,i.item_name as item, sum(o.quantity) 
             from customers as c  
             join sales as s on  c.customer_id = s.customer_id 
             join orders as o on s.sales_id = o.sales_id  
             join items as i on i.item_id = o.item_id  
             where age between 18 and 35 and quantity is not null or 0  group by customer,item '''

In [36]:
cur.execute(command)


In [37]:

with open('./output_sql.csv', 'w', newline='') as file:
    writer = csv.writer(file, delimiter=';')
    writer.writerow(['customer_id', 'item', 'total_quantity'])
    for row in cur.fetchall():
        writer.writerow(row)

In [38]:
#commit and close the connection
connection.commit()
connection.close()

# Solution 2 using python pandas


In [39]:
import pandas as pd
import sqlite3 as s

connection = s.connect('C:\\Users\\Dinesh ram\\Downloads\\Data Engineer - Assignment Database.db')

In [40]:
list_of_tables = ['customers','items','sales','orders']
query_c = 'select * from customers'
query_i = 'select * from items'
query_s = 'select * from sales'
query_o = 'select * from orders'

df_customer = pd.read_sql(query_c,connection)
df_items = pd.read_sql(query_i,connection)
df_sales = pd.read_sql(query_s,connection)
df_orders = pd.read_sql(query_o,connection)

In [41]:
df_customer.head(10)

,customer_id,age
0,1,38
1,2,43
2,3,13
3,4,39
4,5,18
5,6,78
6,7,30
7,8,70
8,9,25
9,10,76


In [42]:
#checking the null values count from each dataframe
df_list=[df_customer,df_items,df_sales,df_orders]

for i in df_list:
    print(i.shape)

(100, 2)
(3, 2)
(500, 2)
(1500, 4)


In [43]:
#checking the null values count from each dataframe
df_list=[df_customer,df_items,df_sales,df_orders]

for i in df_list:
    print(i.isnull().sum()) 

customer_id    0
age            0
dtype: int64
item_id      0
item_name    0
dtype: int64
sales_id       0
customer_id    0
dtype: int64
order_id      0
sales_id      0
item_id       0
quantity    553
dtype: int64


In [44]:
#count of each age number
df_customer['age'].value_counts()


38    6
55    4
30    4
17    3
15    3
69    3
19    3
54    3
50    3
70    3
78    3
13    3
53    2
61    2
43    2
79    2
23    2
58    2
56    2
74    2
73    2
37    2
71    2
72    2
39    2
59    2
22    2
21    2
42    2
14    2
76    2
18    2
65    1
47    1
57    1
51    1
35    1
33    1
45    1
26    1
44    1
29    1
32    1
60    1
52    1
25    1
24    1
31    1
20    1
66    1
41    1
Name: age, dtype: int64

In [45]:
#total number of unique sales_id
len(df_sales['sales_id'].unique())

500

In [49]:

#merge the dataframes 
merged_df = df_customer.merge(df_sales, on='customer_id').merge(df_orders, on='sales_id').merge(df_items, on='item_id')

#filter customers aged 18-35 and calculate the total quantities
result_df = merged_df[(merged_df['age'] >= 18) & (merged_df['age'] <= 35)].groupby(['customer_id','age','item_name'])['quantity'].sum().reset_index()


# remove items with total quantity of 0
result_df = result_df[result_df['quantity'] > 0]

# convert quantities to integers to remove decimal points
result_df['quantity'] = result_df['quantity'].astype(int)

print(result_df)




    customer_id  age item_name  quantity
0             5   18         x         3
1             5   18         y         3
2             5   18         z         4
3             7   30         x        14
4             7   30         y        19
..          ...  ...       ...       ...
67           97   23         y         2
68           97   23         z         3
69          100   30         x         1
70          100   30         y         1
71          100   30         z         5

[70 rows x 4 columns]


In [54]:
# export result to csv with delimiter ';'
result_df.to_csv('./output_pandas.csv', sep=';', index=False)